### *Maciej Pawlikowski*

# Eksploracja danych: system rekomendujący

## Dane

Celem projektu było stworzenie systemu polecającego filmy. Wszystkie eksperymenty prowadziłem na danych MovieLens https://grouplens.org/datasets/movielens/:
- MovieLens Latest Small: http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
- MovieLens 1M Dataset: http://files.grouplens.org/datasets/movielens/ml-1m.zip
- MovieLens 10M Dataset: http://files.grouplens.org/datasets/movielens/ml-10m.zip

Dane zawierają oceny przyznane filmom przez użytkowników:
- Small: 9000 filmów, 700 użytkowników
- 1M: 6000 filmów, 4000 użytkowników
- 10M: 10000 filmów, 72000 użytkowników

## Podejście

Problem znalezienia filmu do polecenia użytkownikowi X można sprowadzić do problemu znalezienia oszacowań brakujących ocen tego użytkownika. Na podstawie ocen, które X wystawił, oraz ocen innych użytkowników znajdujemy przybliżone oceny filmów, których X nie ocenił.

Przetestowałem kilka metod szacujących oceny:
- Baseline Predictors
- User-User Collaborative Filtering
- Item-Item Collaborative Filtering
- Basic Slope One
- Weighted Slope One
- Bi-polar Slope One

Informacje o trzech pierwszych czerpałem z [1], o Slope One przeczytałem w [2].

[1] *Collaborative Filtering Recommender Systems*, Michael D. Ekstrand, John T. Riedl, Joseph A. Konstan http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf

[2] *Slope One Predictors for Online Rating-Based Collaborative Filtering*, Daniel Lemire, Anna Maclachlan http://lemire.me/fr/documents/publications/lemiremaclachlan_sdm05.pdf

## Opis metod

### Baseline Predictors

Dla każdego użytkownika $u$ i filmu $i$ jako przybliżenie oceny $i$ przez $u$ przyjmujemy

$$b_{u,i} = \mu + b_u + b_i,$$
gdzie
$$b_u = \frac{1}{|I_u|} \sum\limits_{i \in I_u} (r_{u,i} - \mu)$$
$$b_i = \frac{1}{|U_i|} \sum\limits_{u \in U_i} (r_{u,i} - b_u - \mu)$$
$\mu$ = średnia wszystkich ocen w zbiorze danych

$I_u$ = przedmioty ocenione przez $u$

$U_i$ = użytkownicy, którzy ocenili $i$

$r_{u,i}$ = ocena wystawiona $i$ przez $u$

$b_{u,i}$ to zatem średnia ocena znieształcona przez odchylenie od średniej właściwe ocenom wystawionym przez $u$ i ocenom wystawionym $i$.

### User-User Collaborative Filtering

Metoda szacuje ocenę na podstawie ocen wystawionych przez podobnych użytkowników. Najpierw obliczamy macierz podobieństw między użytkownikami. Przetestowałem dwie miary podobieństwa:
- miara kosinusowa: $$s(u,v) = \frac{\textbf{r}_u \cdot \textbf{r}_v}{||\textbf{r}_u||_2 ||\textbf{r}_v||_2}$$

  Można też wziąć pod uwagę fakt, że różni użytkownicy mają różne skale oceniania (np. ktoś w ogóle nie używa najwyższej oceny) odejmując od wszystkich ocen średnią danego użytkownika. Wtedy np. trójka wystawiona przez pesymistę może być blisko czwórki przeciętnego użytkownika.
  
  
- korelacja Pearsona: $$s(u,v) = \frac{\sum\limits_{i \in I_u \cap I_v} (r_{u,i}-\bar{r}_u)(r_{v,i}-\bar{r}_v)}{\sqrt{\sum\limits_{i \in I_u \cap I_v} (r_{u,i}-\bar{r}_u)^2} \sqrt{\sum\limits_{i \in I_u \cap I_v} (r_{v,i}-\bar{r}_v)^2}}$$

  Żeby uwiarygodnić nieco tę miarę wynik można dodatkowo uzależnić od liczby przedmiotów ocenionych przez obu użytkowników. Wtedy $s(u,v)$ mnożymy przez $\min(\frac{|I_u \cap I_v|}{threshold}, 1)$. W [1] proponują $threshold = 50$.
  
Po obliczeniu macierzy podobieństw wyznaczamy oceny w następujący sposób:

$$p_{u,i} = \bar{r}_u + \sigma_u \frac{\sum\limits_{u' \in N} s(u,u')(r_{u',i}-\bar{r}_{u'}) / \sigma_{u'}}{\sum\limits_{u' \in N} |s(u,u')|},$$
gdzie 

$\sigma_u$ = odchylenie standardowe ocen użytkownika $u$

$N$ = sąsiedztwo użytkownika $u$.

A zatem szacowana ocena opiera się na średniej ważonej ustandaryzowanych ocen bliskich najbliższych użytkowników. Stosując tę metodę za $N$ biorę pewne $k$ najbliższych $u$ użytkowników spośród takich, którzy ocenili przedmiot $i$.

### Item-Item Collaborating Filtering

Metoda szacuje ocenę wystawioną przez $u$ filmowi $i$ na podstawie ocen wystawionych przez $u$ filmom podobnym do $i$. Jak poprzednio, obliczamy macierz podobieństwa, tym razem dla filmów. Tutaj stosuję tylko miarę kosinusową (wg. [1] współczynnik Pearsona nie sprawdza się tutaj tak dobrze).

$$s(i,j) = \frac{\textbf{r}_i \cdot \textbf{r}_j}{||\textbf{r}_i||_2 ||\textbf{r}_j||_2}$$

Żeby lepiej porównywać różne skale oceniania różnych użytkowników, na czas obliczania podobieństw od wszystkich ocen możemy najpierw odjąć oceny bazowe $b_{u,i}$. W tabeli wyników ta zmiana jest reprezentowana przez "-baseline".

Szacujemy oceny tak:
$$p_{u,i} = \frac{\sum\limits_{j \in S} s(i,j)(r_{u,j}-b_{u,i})}{\sum\limits_{j \in S} |s(i,j)|} + b_{u,i},$$
gdzie

$S$ = pewne $k$ najbliższych filmowi $i$ filmów, spośród tych, które zostały ocenione przez $u$.

### Schematy Slope One

Schematy Slope One opierają się na "przesuwaniu" ocen. Próbując oszacować ocenę filmu $i$ dla użytkownika $u$ bierzemy pod uwagę średnie różnice między ocenami filmów ocenionych przez $u$ a ocenami filmu $i$.

Najpierw obliczamy macierz średnich różnic między ocenami filmów:

$$dev_{j,i} = \sum\limits_{u \in I_{j,i}} \frac{u_j - u_i}{|I_{j,i}|},$$
gdzie

$I_{j,i}$ = użytkownicy, którzy ocenili zarówno film $j$, jak i film $i$.


### Basic Slope One

Dla użytkownika $u$ ocenę filmu $j$ na podstawie oceny filmu $i$ można szacować jako $dev_{j,i} + u_i$. Podstawowy schemat Slope One to średnia z takich oszacowań:

$$p_{u,j} = \frac{1}{|R_j|} \sum\limits_{i \in R_j} (dev_{j,i} + u_i),$$
gdzie

$R_j = \lbrace i | i \in I_u, i \neq j, |I_{j,i}| > 0 \rbrace $.

W [2] jest zaproponowane przybliżenie tej metody, które trochę zmniejsza koszt obliczania ocen. Dla odpowiednio gęstych danych $R_j$ prawie zawsze jest równe $I_u \setminus \lbrace j \rbrace$. W takim przypadku można formułę uprościć:

$$p_{u,j} = \bar{u} + \frac{1}{|R_j|} \sum\limits_{i \in R_j} dev_{j,i}.$$

Nie korzystałem z tego uproszczenia, bo w danych, na których pracowałem, było całkiem sporo dziur. Wydaje mi się, że założenie, że $R_j$ to prawie $I_u \setminus \lbrace j \rbrace$, jest łamane zbyt często, żeby było uzasadnione.

### Weighted Slope One

Ważony Slope One stara się uwiarygodnić te szacowania, które powstały na podstawie ocen większej liczby użytkowników. Elementy macierzy $dev$ są więcej warte, gdy zostały obliczone z pięciuset ocen niż gdy z pięciu. Zamiast zwykłej średniej bierzemy średnią ważoną, z liczbami użytkowników jako wagami:

$$p_{u,j} = \frac{\sum\limits_{i \in S_u \setminus \lbrace j \rbrace} (dev_{j,i} + u_i) |I_{j,i}|}{\sum\limits_{i \in S_u \setminus \lbrace j \rbrace} |I_{j,i}|}$$

### Bi-polar Slope One

Ta metoda dzieli oceny każdego użytkownika na dwie grupy: oceny pozytywne i oceny negatywne. Za pozytywne są uznawane oceny wyższe niż średnia danego użytkownika. Nie wyciągamy żadnych wniosków z różnicy ocen filmu $i$ między użytkownikami $u$ i $v$ gdy $u$ lubi $i$, a $v$ nie. Budujemy dwie osobne macierze średnich różnic, jedną mierzącą różnice między ocenami pozytywnymi, drugą - między negatywnymi:

$$dev_{j,i}^{like} = \sum\limits_{u \in I_{j,i}^{like}} \frac{u_j - u_i}{|I_{j,i}^{like}|},$$
gdzie

$I_{j,i}^{like}$ = użytkownicy, którzy pozytywnie ocenili zarówno film $j$, jak i film $i$.

Analogicznie definiujemy $dev^{dislike}$.

Szacowanie dla użytkownika $u$ oceny filmu $j$ mając jego ocenę filmu $i$ zależy od tego, czy $u$ lubi $i$. Jeśli tak, to jest to $dev_{j,i}^{like} + u_i$, a jeśli nie, $dev_{j,i}^{dislike} + u_i$. Pełny wzór wygląda tak:

$$p_{u,j} = \frac{{\sum\limits_{i \in I_u^{like} \setminus \lbrace j \rbrace} (dev_{j,i}^{like} + u_i) |I_{j,i}^{like}|} + {\sum\limits_{i \in I_u^{dislike} \setminus \lbrace j \rbrace} (dev_{j,i}^{dislike} + u_i) |I_{j,i}^{dislike}|}}{{\sum\limits_{i \in I_u^{like} \setminus \lbrace j \rbrace} |I_{j,i}^{like}|} + {\sum\limits_{i \in I_u^{dislike} \setminus \lbrace j \rbrace} |I_{j,i}^{dislike}|}}.$$

## Testowanie

Zastosowałem metodę ewaluacji systemu opartą na walidacji krzyżowej:

1. Losowo posortuj dane (macierz $X$ o kształcie (liczba użytkowników x liczba filmów)).
2. Podziel dane na 10 równych części $X_1 \dots X_{10}$ ($X_1$ to pierwsze $1/10$ wierszy itd.).
3. Dla każdej części $X_i$:
    - ukryj losowo wybraną połowę ocen każdego użytkownika w $X_i$
    - naucz system na pozostałych dziewięciu częściach
    - na podstawie widocznych ocen w $X_i$ oszacuj oceny ukryte
    - sprawdź jak szacowania mają się do ocen ukrytych za pomocą pewnej miary $d$
4. Zwróć średnią i odchylenie standardowe wartości $d$ na wszystkich dziesięciu częściach

Używałem dwóch miar jakości oszacowań:
$$MAE = \frac{1}{n} \sum\limits_{u,i} |r_{u,i} - p_{u,i}|$$

$$RMSE = \sqrt{\frac{1}{n} \sum\limits_{u,i} (r_{u,i} - p_{u,i})^2},$$

gdzie $n$ to liczba ukrytych ocen.

## Implementacja

Implementacja Baseline Predictors jest oczywista.

### Uwagi ogólne (do wszytkich następnych metod)

Może się zdarzyć, że nie będzie się dało oszacować oceny danego filmu (bo na przykład nikt oprócz testowanego użytkownika go nie ocenił). W takich przypadkach używam Baseline Predictors do zapełnienia luk w tabeli.

Przy testowaniu obliczamy oceny dla wielu użytkowników naraz, ale przy każdym z nich interesują nas bardzo specyficzne przedmioty (kilkanaście, kilkadziesiąt przedmiotów dla jednego użytkownika, różnych dla różnych użytkowników). Trudno jest obliczyć szacowania za pomocą operacji macierzowych, z dwóch powodów:
  - powstałe macierze pośrednie miałyby dodatkowy wymiar i byłyby bardzo duże;
  - obliczalibyśmy dużo więcej ocen niż trzeba, bo nie moglibyśmy się ograniczyć tylko do tych kilkunastu-kilkudziesięciu filmów na użytkownika.
  
W związku z tym obliczam odległości globalnie, macierzowo, ale szacowania ocen robię w pętli for. Testowałem wersję z większymi macierzami i przetwarzaniem użytkowników w porcjach, ale działało to wolniej, bo obliczałem znacznie więcej ocen niż trzeba. To rozwiązanie ma znaczenie tylko przy testowaniu, gdy szacujemy wybrane oceny wielu użytkowników naraz.

W niektórych metodach proponowane oceny mogą wyjść poza skalę. Przy polecaniu filmów się tym nie przejmuję, bo dalej definiują one pewien porządek na filmach, ale przy ocenianiu systemu jest to problem. Dlatego podczas ewaluacji obcinam oceny do właściwego zakresu (np. $6,23$ jest traktowane jako $5,0$).

Podczas polecania filmów użytkownikowi nie wybieram oczywiście tych, które już ocenił.

### User-User Collaborative Filtering

Drobna zmiana ocen pojedynczego użytkownika może znacznie zaburzyć jego odległości od pozostałych, więc podobieństwo oblicza się tutaj na etapie rekomendacji (wstępnie obliczona macierz podobieństw użytkowników szybko się dezaktualizuje). W związku z tym ta metoda ma najwolniejszy proces polecania filmu pojedynczemu użytkownikowi.

Odległości kosinusowe obliczam funkcją $scipy.spatial.distance.cdist$, korelację Pearsona ręcznie. Przydaje się tutaj (i później) następująca zależność: żeby dostać sumę ocen $u$, ale tylko tych filmów, które również ocenił $v$, wystarczy zrobić iloczyn skalarny $u$ z maską binarną $(v>0)$.

Żeby znaleźć najbliższych $k$ użytkowników, którzy ocenili film $i$ zamieniam zera w wektorze ocen $i$ w miejscach odpowiadających użytkownikom, którzy nie ocenili $i$ na wartość $nan$ (która zawsze jest na końcu po sortowaniu w pythonie). Tak zmodyfikowany wektor ocen $i$ stanowi pojedynczą kolumnę macierzy, na której pracuję. Jest to przydatne, bo dla każdego filmu liczba znalezionych sąsiadów może być inna (nie wszystkie filmy mają odpowiednio dużo ocen). Następnie robię $numpy.argpartition$ na kolumnach i wycinam odpowiedni fragment. Potem zostaje już właściwie tylko suma ważona po kolumnach.

### Item-Item Collaborative Filtering

W tej metodzie (i wszystkich następnych) odległości są obliczane między filmami, więc są znacznie mniej czułe na zmiany ocen pojedynczych użytkowników. Dzięki temu można je obliczyć raz, a później tylko co jakiś czas aktualizować, co sprawia, że rekomendację są dość szybkie. Przy testowaniu, przynajmniej moją metodą, nie robi to różnicy.

Obliczanie ocen jest bardzo podobne do poprzedniej metody.

### Schematy Slope One

Podstawowym wyzwaniem obliczeniowym w Slope One jest efektywne obliczenie macierzy $dev$. Dalsze etapy we wszystkich odmianach tej metody opierają się na prostym indeksowaniu i sumowaniu macierzy ocen użytkownika, macierzy $dev$ i macierzy $cards$, gdzie ${cards}_{j,i} = |I_{j,i}|$. Dzięki temu, że nie szukamy tutaj najbliższych sąsiadów, matody Slope One mają najkrótszy czas obliczania rekomendacji ze wszystkich testowanych metod (oprócz oczywiście Baseline Predictors).

Dla przypomnienia:

$$dev_{j,i} = \sum\limits_{u \in I_{j,i}} \frac{u_j - u_i}{|I_{j,i}|}.$$

Do obliczenia $dev$ korzystam z zależności opisanej w sekcji o User-User CF (o iloczynie skalarnym z maską binarną). Możemy osobno obliczyć sumę ocen wchodzących do mianownika dla $j$ i dla $i$, a później zrobić jedno odejmowanie i podzielić przez ${cards}_{j,i}$. Prowadzi to do następującego kodu:

In [1]:
def calculate_similarities_SO(scores, nan_to_0=True):
    rated = scores > 0
    dists = scores.T.dot(rated)
    dists = dists - dists.T
    cards = rated.T.dot(rated.astype(np.float))
    return np.nan_to_num(dists / cards), cards

Funkcja $calculate\_similarities\_SO$ zwraca parę $(dev, cards)$. Rzutowanie na $numpy.float$ pozwala dostać wartości liczbowe (normalnie iloczyn skalarny wektorów boolowskich zwraca wartość boolowską). Gdy pewne dwa filmy nie zostały razem ocenione, ustawiam ich podobieństwo na $0$.

## Wyniki testów

Duże czasy działania testów niektórych metod na największych danych wynikają z przekroczenia dostępnej pamięci podczas obliczania macierzy podobieństw między filmami. Nie optymalizowałem tego, bo te macierze oblicza się raz, a później tylko co jakiś czas poprawia. Wielokrotne ich obliczanie jest potrzebne tylko do ewaluacji systemu, co z kolei (w moim przypadku) również jest jednorazową akcją. W prawdziwym systemie lepiej jest oczywiście bardziej oszczędzać pamięć, ale niestety na to nie starczyło mi już czasu.

W przypadku User-User CF testowanie zajęło najdłużej. Wynikało to z konieczności szukania najbliższych sąsiadów w bardzo dużym zbiorze (użytkowników, nie filmów). Proces ten można zrównoleglić i przetwarzać kilku użytkowników naraz, ale ja tego nie zrobiłem.

W tabeli wyniki są zaprezentowane w postaci

    [średnia MAE      średnia RMSE   ]
    [odch. std. MAE   odch. std. RMSE]
    czas testu

## Wnioski

Wyniki Baseline Predictors zgodnie z przewidywaniami wypadły najgorzej, ale sama metoda przydała jako uzupełnienie pozostałych algorytmów.

Ustawienie parametru $threshold$ w User-User CF wyraźnie zwiększyło dokładność. Po tym usprawnieniu odległość kosinusowa i korelacja Pearsona osiągnęły bardzo podobne wyniki. Wartość $k$ nie miała na tyle dużego wpływu na dokładność, jak się spodziewałem. Wartości z zakresu $[10,50]$ dawały podobne rezultaty.

W przypadku Item-Item CF odjęcie szacowań bazowych znacznie poprawiło wyniki na dużych danych. Z pewnych przyczyn na małym zbiorze efekt był dokładnie odwrotny. Nie jestem pewien z czego to może wynikać. Podobnie jak poprzednio, zakres rozsądnych wartości $k$ jest dość szeroki.

Metody Slope One mają za zadanie nie tyle zwiększenie dokładności systemu, co jego uproszczenie z zachowaniem dobrej skuteczności. W eksperymencie poradziły sobie dobrze na małych danych: błędy nie odbiegały wiele od błędów metod CF, a czas szacowania ocen pojedyńczego użytkownika został znacznie zmniejszony. Na dużych danych niestety Slope One okazało się niewiele lepsze of trywialnego Baseline Predictors, podczas gdy metody CF zadziałały wyraźnie lepiej.

Ważona wersja Slope One nieznacznie polepszyła dokładność, wersja z podziałem na przedmioty lubiane i nielubiane w dwóch z trzech przypadków zadziałała odwrotnie. Może to być efekt rozrzedzenia już i tak niezbyt gęstego zestawu danych, bo na dużych danych ta metoda przyniosła minimalną poprawę.

Jeśli chodzi o możliwe ulepszenia, to oprócz wspomnianych wcześniej optymalizacji można spróbować technik redukcji wymiarowości wektorów ocen. W ten sposób macierze podobieństwa mogłyby być obliczane jeszcze szybciej. Grupowanie klientów pomogłoby skrócić czas działania Collaborative Filtering w wersji User-User. 